In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
import numpy as np

import os
import cProfile
from tqdm import tqdm

In [ ]:
ws = sd.WhoScored(
    #  "GER-Bundesliga", "GER-Bundesliga2", "ENG-Premier League", "ESP-La Liga", "FRA-Ligue 1", "ITA-Serie A"
    leagues=["GER-Bundesliga2", "GER-Bundesliga", "ENG-Premier League"],
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

In [ ]:
schedule = ws.read_schedule().reset_index()
test_game = schedule["game_id"].values[0]
test_date = schedule[schedule["game_id"] == test_game]["date"].values[0]

In [ ]:
schedule

---

In [ ]:
# get dataset
import gde.database_io.dims
import gde.database_io.faks

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

test_game = 1557933
db_path = "/home/morten/Develop/packing-report/gde/GDE.db"

engine = create_engine(f'sqlite:///{db_path}', echo=False)
session = Session(engine)


from sqlalchemy import func

from sqlalchemy import func
from sqlalchemy.orm import aliased

# Subquery for result_table
ranked_subquery = (
    session.query(
        gde.database_io.dims.Games.game_id,
        gde.database_io.dims.Games.team_id,
        gde.database_io.dims.Games.result,
        func.row_number().over(
            partition_by=(gde.database_io.dims.Games.game_id, gde.database_io.dims.Games.team_id),
            order_by=func.count().desc()
        ).label('rank')
    )
    .filter(gde.database_io.dims.Games.game_date > '2018-06-01')
    .group_by(gde.database_io.dims.Games.game_id, gde.database_io.dims.Games.team_id, gde.database_io.dims.Games.result)
    .subquery()
)

result_table = (
    session.query(ranked_subquery.c.game_id, ranked_subquery.c.team_id, ranked_subquery.c.result)
    .filter(ranked_subquery.c.rank == 1)
    .subquery()
)

# Subquery for elo_table
elo_table = (
    session.query(
        func.avg(gde.database_io.dims.Games.elo).label('avg_elo'),
        gde.database_io.dims.Games.team_id,
        gde.database_io.dims.Games.game_id
    )
    .filter(gde.database_io.dims.Games.game_date > '2018-06-01')
    .group_by(gde.database_io.dims.Games.game_id, gde.database_io.dims.Games.team_id)
    .subquery()
)

# Joining the subqueries
query = (
    session.query(
        result_table.c.game_id,
        result_table.c.team_id,
        result_table.c.result,
        elo_table.c.avg_elo
    )
    .join(elo_table, 
          (result_table.c.game_id == elo_table.c.game_id) & 
          (result_table.c.team_id == elo_table.c.team_id))
)

results = query.all()

In [ ]:
# get dataset
import gde.database_io.dims
import gde.database_io.faks

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

test_game = 1557933
db_path = "/home/morten/Develop/packing-report/gde/GDE.db"

engine = create_engine(f'sqlite:///{db_path}', echo=False)
session = Session(engine)


from sqlalchemy import func

from sqlalchemy import func
from sqlalchemy.orm import aliased

# Subquery for result_table
ranked_subquery = (
    session.query(
        gde.database_io.dims.Games.game_id,
        gde.database_io.dims.Games.team_id,
        gde.database_io.dims.Games.result,
        func.row_number().over(
            partition_by=(gde.database_io.dims.Games.game_id, gde.database_io.dims.Games.team_id),
            order_by=func.count().desc()
        ).label('rank')
    )
    .filter(gde.database_io.dims.Games.game_date > '2018-06-01')
    .group_by(gde.database_io.dims.Games.game_id, gde.database_io.dims.Games.team_id, gde.database_io.dims.Games.result)
    .subquery()
)

result_table = (
    session.query(ranked_subquery.c.game_id, ranked_subquery.c.team_id, ranked_subquery.c.result)
    .filter(ranked_subquery.c.rank == 1)
    .subquery()
)

# Subquery for elo_table
elo_table = (
    session.query(
        func.avg(gde.database_io.dims.Games.elo).label('avg_elo'),
        gde.database_io.dims.Games.team_id,
        gde.database_io.dims.Games.game_id
    )
    .filter(gde.database_io.dims.Games.game_date > '2018-06-01')
    .group_by(gde.database_io.dims.Games.game_id, gde.database_io.dims.Games.team_id)
    .subquery()
)

# Joining the subqueries
query = (
    session.query(
        result_table.c.game_id,
        result_table.c.team_id,
        result_table.c.result,
        elo_table.c.avg_elo
    )
    .join(elo_table, 
          (result_table.c.game_id == elo_table.c.game_id) & 
          (result_table.c.team_id == elo_table.c.team_id))
)

results = query.all()

In [ ]:
pred_data = pd.DataFrame(np.array(results), columns=["game_id", "team_id", "result", "avg_elo"])

In [ ]:
pred_data = pd.merge(pred_data, pred_data, how="outer", on="game_id")
pred_data = pred_data.loc[pred_data["team_id_x"] != pred_data["team_id_y"]]

In [ ]:
pred_data

In [ ]:
pred_data["elo_diff"] = pred_data["avg_elo_x"].astype(float) - pred_data["avg_elo_y"].astype(float)
pred_data["result_diff"] = pred_data["result_x"].apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
#sns.distplot(pred_data["result_diff"])
stats.probplot(pred_data["result_diff"], plot=plt)

In [ ]:
import plotly.express as px

# feat = np.log1p(pred_data["result_diff"])

px.scatter(pred_data, x="result_diff", y="elo_diff")

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
lr.fit(X=pred_data["elo_diff"].values.reshape(-1, 1), y=pred_data["result_diff"])

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
lr.predict([[1676.1701826960673 - 1780.7405858122504]])

In [ ]:
pred_data = pred_data.groupby('game_id').first().reset_index()

In [ ]:
pred_data

In [ ]:
pred_data["prediction"] = pred_data.apply(lambda x: lr.predict([[x["elo_diff"]]])[0], axis=1)

In [ ]:
pred_data

In [ ]:
#sns.distplot(pred_data["result_diff"])
stats.probplot(pred_data["result_diff"], plot=plt)

In [ ]:
np.mean((pred_data["result_diff"] - pred_data["prediction"])**2)